# Schreier-Sims for Rubik's Cubes

In [5]:
import numpy as np, time, random
from sympy.combinatorics import Permutation
from functions import enumerate_cube, get_human_order, get_human_order_multiple, get_solved_states, compute_schreiers_tree, compute_generating_set, apply_sims_filter
import config

# choose size of Rubik's cube
n = 3

# N: order of the symmetric group (number of enumerated stickers)
if n % 2 == 0:
    N = 6 * n**2
else:
    N = 6 * n**2 - 6

# set of moves that generate the Rubik's cube
generators = enumerate_cube(n, N)

identity_order = range(N)
random_order = [(x,) for x in random.sample(range(N), N)]
human_order = get_human_order(n)
human_order_multiple = get_human_order_multiple(n)
# choose solving order
solving_order = random_order

# list of group orders of the stabilizer groups
orders_list = []

# list of generating sets of the stabilizer groups
generators_list = []
length_generators_list_before_sims_filter = []
length_generators_list = []

# list of Schreiers vectors for the stabilizer groups
schreiers_tree_list = []

# dictionary of lengths of moves written as composition of original generators
config.lengths = {Permutation(N-1):0}
for generator in generators:
    config.lengths[generator] = 1

# the algorithm
i = 0

length_generators_list_before_sims_filter.append(len(generators))

temp_generators = apply_sims_filter(generators, N)
length_generators_list.append(len(temp_generators))
generators_list.append(temp_generators)

start_time = time.time()

while len(temp_generators) > 0:
    k = solving_order[i]

    schreiers_tree = compute_schreiers_tree(temp_generators, N, k)
    schreiers_tree_list.append(schreiers_tree)

    orders_list.append(len(schreiers_tree))

    new_generators = compute_generating_set(temp_generators, schreiers_tree, k)
    length_generators_list_before_sims_filter.append(len(new_generators))

    temp_generators = apply_sims_filter(new_generators, N)
    length_generators_list.append(len(temp_generators))
    generators_list.append(temp_generators)

    i += 1

    elapsed_time = time.time()-start_time
    print(f"Time after iteration {i}: {elapsed_time:.2f} seconds")


# compute visually solved states (useful for membership testing)
solved_states = get_solved_states(n, N)

Time after iteration 1: 0.00 seconds
Time after iteration 2: 0.36 seconds
Time after iteration 3: 0.85 seconds
Time after iteration 4: 1.45 seconds
Time after iteration 5: 1.46 seconds
Time after iteration 6: 1.76 seconds
Time after iteration 7: 1.76 seconds
Time after iteration 8: 1.76 seconds
Time after iteration 9: 1.77 seconds
Time after iteration 10: 1.97 seconds
Time after iteration 11: 2.25 seconds
Time after iteration 12: 2.58 seconds
Time after iteration 13: 2.58 seconds
Time after iteration 14: 2.66 seconds
Time after iteration 15: 2.80 seconds
Time after iteration 16: 2.87 seconds
Time after iteration 17: 2.92 seconds
Time after iteration 18: 2.94 seconds
Time after iteration 19: 2.94 seconds
Time after iteration 20: 2.94 seconds
Time after iteration 21: 2.94 seconds
Time after iteration 22: 2.96 seconds
Time after iteration 23: 2.97 seconds
Time after iteration 24: 2.97 seconds
Time after iteration 25: 2.97 seconds
Time after iteration 26: 2.97 seconds
Time after iteration 

## Order of Rubik's cube group

In [7]:
orders_list = np.array(orders_list, dtype=object)
print(np.prod(orders_list))

43252003274489856000


## Membership testing and length of solving algorithm

In [12]:
# choose the element you want to perform the check on

# some examples of moves:

# for generator in generators:
#     move = Permutation(generator)
#     print(generator)
l = list(generators)
move = l[1]*l[2]*l[3]*(l[2]**-1)*l[5]*l[1]*(l[3]**-1)*l[4]*(l[5]**-1)*l[0]*(l[4]**-1)*l[5]
print(move)
# move = Permutation(N-1)(0,10,20)(6,12,40) # double corner twist on 3x3
# move = Permutation(N-1)(51,63)(0,40,20)(32,52,72) # double corner twist and enter swap on 4x4

# the algorithm

contained = False
length_solving_algorithm = 0

for solved_state in solved_states:
    g = move * solved_state
    length_solving_algorithm = 0

    i = 0
    while i < len(schreiers_tree_list):
        k = solving_order[i]

        schreiers_tree = schreiers_tree_list[i]
        h = tuple(g(x) for x in k)
        u = schreiers_tree.get(h)
        if u is None:
            break
        else:
            g = g * u**-1
            length_solving_algorithm += config.lengths[u]
        i += 1

    if g == Permutation(N-1):
        contained = True
        print(length_solving_algorithm)
        break

print(contained)

(0 28 6 26 24 20 8 12 16 38 10 22 40 36 44)(1 45)(3 33)(4 42 32)(5 43 47 35 7)(9 21)(11 41 39 13 17)(14 30 46)(19 31)(23 37)(25 27)
1596065754
True
